# Simulation Truth

This notebook will introduce you to the concept of simulation truth in fuse.



## Imports and Simulation Context

Similar to the previous notebooks, we will start by importing the necessary modules and creating a simulation context. Additional we register two new plugins called `PeakTruth` and `SurvivingClusters`.

In [1]:
import fuse
import numpy as np

In [ ]:
st = fuse.context.full_chain_context(output_folder = "./fuse_data")

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "entry_stop": 10,
               })

run_number = "00000"

## Raw_Records and Contributing_Clusters

First we will run the simulation up to `raw_records`. The `PMTResponseAndDAQ` plugin now has two outputs, `raw_records` and `contributing_channels`, both are saved to disk when we request fuse to produce `raw_records`.

In [3]:
st.make(run_number, "microphysics_summary")
st.make(run_number, "raw_records")

/home/hschulze/.local/lib/python3.9/site-packages/straxen/url_config.py:743: UserWarning: From straxen version 2.1.0 onward, URLConfig parameterswill be sorted alphabetically before being passed to the plugins, this will change the lineage hash for non-sorted URLs. To load data processed with non-sorted URLs, you will need to use an older version.
  warnings.warn(


Now that the data is produced, lets load it. Both are of the same `data_kind` so we can load them together.

In [4]:
raw_records = st.get_array(run_number, ["raw_records", "contributing_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

`contributing_clusters` gives you five additional columns. These are:
- `contributing_clusters` - A list of the clusters that contributed to the `raw_record`
- `s1_photons_per_cluster` - The number of S1 photons that of the corresponding cluster in the `raw_record`
- `s2_photons_per_cluster` - The number of S2 photons that of the corresponding cluster in the `raw_record`
- `ap_photons_per_cluster` - The number of (virtual) PMT afterpulse 'photons'
- `raw_area` - The sum of the contributing photon gains divided by the gain of the PMT

Lets have a look what clusters contributed to the first record:

In [5]:
print(raw_records[0]["contributing_clusters"])

[1 0 0 0 0]


You can see that we get a list of length 5. This is a compromise we need to make as we can't store a list of variable length in a strax. In this case we only store the information of the 5 first clusters that contributed to the record. If there are more than 5 clusters for one record, this information is lost. For simulations with a lot of clusters per event, it makes sense to increase the number of clusters that are stored per record. This can be done by changing the config option `max_contributing_channels_in_truth` of the `PMTResponseAndDAQ` plugin. Lets try this out:

In [6]:
st.set_config({"max_contributing_channels_in_truth": 35,})
st.make(run_number, "raw_records")

In [7]:
raw_records = st.get_array(run_number, ["raw_records", "contributing_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

In [8]:
index = np.argmax(np.sum(raw_records["contributing_clusters"]>0, axis = 1))

print(raw_records[index]["contributing_clusters"])

[135 104 133 127  20  55  97 103 124  44  72  66  54  53  74  42  41  34
  33  31  27  21  19  17  90 136 187 184 174 158 154 147 138 131  75]


Now the list is 35 elements long, just as we requested. Depending on the source this might still be not enough. We can now have a look with how many photons each of these clusters contributed to the record:

In [9]:
print("S1 photons:", raw_records[index]["s1_photons_per_cluster"])
print("S2 photons:", raw_records[index]["s2_photons_per_cluster"])
print("AP photons:", raw_records[index]["ap_photons_per_cluster"])

S1 photons: [4 3 3 3 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
S2 photons: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
AP photons: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


We can have a look at the clusters that contributed to the record. `microphysics_summary` contains the column `cluster_id`. This is the same number as stored in `contributing_clusters`. Please note that `cluster_id` is only unique per chunk of data.

In [10]:
microphysics_summary = st.get_df(run_number, "microphysics_summary")

Loading microphysics_summary: |          | 0.00 % [00:00<?]

In [11]:
microphysics_summary[np.isin(microphysics_summary.cluster_id.values, raw_records[index]["contributing_clusters"])].head()

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,...,x_pri,y_pri,z_pri,cluster_id,xe_density,vol_id,create_S2,photons,electrons,excitons
4,27,2827386342,2827386342,-36.266766,-21.600817,-9.273911,67.258522,8,0,0,...,-44.154747,2.803934,7.933488,124,2.862,1,True,4160,766,733
19,24,2827386343,2827386343,-12.580804,17.099777,-36.494366,215.079941,8,0,0,...,-44.154747,2.803934,7.933488,17,2.862,1,True,11213,4512,2368
21,24,2827386343,2827386343,-12.649805,17.200773,-36.587608,378.983673,8,0,0,...,-44.154747,2.803934,7.933488,19,2.862,1,True,18687,9338,4344
22,24,2827386343,2827386343,-12.725284,17.268085,-36.636372,415.224915,8,0,0,...,-44.154747,2.803934,7.933488,20,2.862,1,True,17571,12975,4617
23,24,2827386343,2827386343,-12.798026,17.312786,-36.671024,385.584381,8,0,0,...,-44.154747,2.803934,7.933488,21,2.862,1,True,19036,9534,4449


## Peaks and peak_truth

Next we can process the simulation result to `peak_basics`. Strax(en) will merge multiple records into a peak. The PeakTruth plugin will evaluate which `raw_records` contribute to a peak and calculate a truth output for each peak. The provided columns for each peak are:
- `s1_photon_number_truth` - The number of S1 photons that contributed to the peak
- `s2_photon_number_truth` - The number of S2 photons that contributed to the peak
- `ap_photon_number_truth` - The number of (virtual) PMT afterpulse 'photons' that contributed to the peak
- `raw_area_truth` - The sum of all contributing photon gains divided by the gains of the PMTs
- `observable_energy_truth` - Estimate of the energy that is associated with the peak.
- `number_of_contributing_clusters` - Number of clusters that contributed to the peak
- `average_x_of_contributing_clusters` - Weighted average of the x position of the clusters that contributed to the peak
- `average_y_of_contributing_clusters` - Weighted average of the y position of the clusters that contributed to the peak
- `average_z_of_contributing_clusters` - Weighted average of the z position of the clusters that contributed to the peak
- `average_x_obs_of_contributing_clusters` - Weighted average of the observed x position of the clusters that contributed to the peak
- `average_y_obs_of_contributing_clusters` - Weighted average of the observed y position of the clusters that contributed to the peak
- `average_z_obs_of_contributing_clusters` - Weighted average of the observed z position of the clusters that contributed to the peak

Lets take a closer look at `observable_energy_truth` using an example: 
If we would have two clusters, the first one with 100 keV energy producig 100 S1 photons and the second one with 10 keV producing 10 S1 photons. After simulation and processing we find two S1 peaks in our data. The first S1 consitis of 90 photons from the first cluster and 5 photons of the second cluster. The `observable_energy_truth` for this peak is calculated as: 90/100 * 100 keV + 5/10 * 10 keV = 90 keV + 5 keV = 95 keV. The second S1 consists of 3 photons from the first cluster and 4 photons of the second cluster. The `observable_energy_truth` for this peak is calculated as: 3/100 * 100 keV + 4/10 * 10 keV = 3 keV + 4 keV = 7 keV. A similar calculation is done for the S2 peaks but replacing the S1 photons with the S2 photons.


In [12]:
st.make(run_number, "peak_truth")
st.make(run_number, "peak_positions")

As strax(en) will take care of the matching of our truth information to the individual peaks, we can simply load the `peak_basics` and `peak_truth` data together.

In [13]:
peak_basics = st.get_df(run_number, ["peak_basics", "peak_truth", "peak_positions"])

Loading plugins: |          | 0.00 % [00:00<?]

For a peak area bias study we could now compare the raw_area to the peak area:

In [14]:
peak_basics[["area", "raw_area_truth"]].head()

,area,raw_area_truth
0,2.808850,3.049999
1,1059.032593,1063.919800
2,61364.210938,61369.085938
3,718.991394,721.309814
4,341264.375000,320888.656250


We might also be interested in the peak classification: 

In [15]:
peak_basics[["type", "s1_photon_number_truth", "s2_photon_number_truth", "ap_photon_number_truth"]].head()

,type,s1_photon_number_truth,s2_photon_number_truth,ap_photon_number_truth
0,1,3,0,0
1,2,0,827,0
2,1,36192,0,36
3,2,0,551,0
4,2,0,251710,250


Or you might want to check how our position reconstruction is doing: 

In [16]:
peak_basics[["type","x","y", "average_x_obs_of_contributing_clusters", "average_y_obs_of_contributing_clusters", "average_z_obs_of_contributing_clusters"]].head()

,type,x,y,average_x_obs_of_contributing_clusters,average_y_obs_of_contributing_clusters,average_z_obs_of_contributing_clusters
0,1,NaN,NaN,-20.890713,-51.367107,-1.464800
1,2,-20.788963,-52.198101,-20.890713,-51.367107,-1.464800
2,1,-57.817341,0.454473,-19.766459,2.009263,-24.624775
3,2,-38.655247,-18.313650,-38.867641,-18.246555,-6.679745
4,2,-37.277332,-18.950422,-34.189152,-14.120831,-11.868514


## Surviving Clusters
Finally we can evaluate if an energy deposit makes it into a record or a peak. This is done by the `SurvivingClusters` plugin. It will provide the following columns:
- `in_a_record` - Boolean if the cluster is in a record
- `in_a_peak` - Boolean if the cluster is in a peak

In [17]:
st.make(run_number, "surviving_clusters")
microphysics_summary = st.get_df(run_number, ["microphysics_summary", "surviving_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

Now that we have the data loaded we could have a look at clusters that did not make it into a peak: 

In [18]:
microphysics_summary.query("in_a_peak == False").head()

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,...,z_pri,cluster_id,xe_density,vol_id,create_S2,photons,electrons,excitons,in_a_record,in_a_peak
170,27,2827386497,2827386497,32.959923,14.185023,-9.616013,0.096808,0,0,0,...,7.933488,171,2.862,1,True,0,0,0,False,False
203,26,2827386535,2827386535,27.667393,10.273334,-12.147082,0.044494,0,0,0,...,7.933488,204,2.862,1,True,0,0,0,False,False
205,25,2827386710,2827386710,9.488111,6.251637,-11.284761,0.141653,0,0,0,...,7.933488,206,2.862,1,True,0,0,0,False,False
208,24,2827386851,2827386851,16.457802,8.368632,-30.002600,0.085379,0,0,0,...,7.933488,209,2.862,1,True,0,0,0,False,False
209,24,2827386893,2827386893,16.093048,14.685450,-34.862846,0.057414,0,0,0,...,7.933488,210,2.862,1,True,0,0,0,False,False


## Event Truth

Finally we can have a look at truth information at the event level. This is done by the `EventTruth` plugin. It will provide the following columns:
- `x_obs_truth` - The x position of the event. This corresponds to the x position of the main S2.
- `y_obs_truth` - The y position of the event. This corresponds to the y position of the main S2.
- `z_obs_truth` - The z position of the event. This is calculated as mean of the main S1 and S2 `average_z_obs_of_contributing_clusters`. Does this make sense?
- `energy_of_main_peak_truth` - This is intended to be the energy that can be found in the main S1 and S2. It is calculated as the mean of the `observable_energy_truth` of the main S1 and S2. Does this make any sense???
- `total_energy_in_event_truth` - The sum of all energy deposits that are in the event

In [19]:
st.make(run_number, "event_truth")

In [20]:
event_data = st.get_df(run_number, ["event_info", "event_truth"])

Loading plugins: |          | 0.00 % [00:00<?]

First lets take a look at the energy informations: 

In [21]:
event_data[["e_ces", "energy_of_main_peak_truth", "total_energy_in_event_truth"]]

,e_ces,energy_of_main_peak_truth,total_energy_in_event_truth
0,0.950759,5.012142,5.101432
1,10316.319336,5868.240723,9708.582031
2,1329.449341,772.092590,1575.655151
3,2615.176758,2229.846436,4019.808838
4,7898.591797,3581.932861,15595.068359
5,17511.164062,750.065796,1264.030273
6,2871.658936,2855.382080,4376.391602
7,1303.236450,1660.800293,1684.907104
8,6.340443,29.601395,31.483326


And the positions: 

In [22]:
event_data[["x", "x_obs_truth", "z", "z_obs_truth"]]

,x,x_obs_truth,z,z_obs_truth
0,-20.420223,-20.890713,-2.169019,-1.464800
1,-13.204618,-29.022295,-34.250954,-19.787865
2,-18.117203,-19.628901,-51.266327,-13.692792
3,-50.167782,-47.092503,-19.874458,-13.965933
4,14.325102,0.000000,-17.188948,-12.487251
5,-8.170411,-19.098410,-115.728760,-122.670464
6,-23.360212,-27.328758,-3.040385,-4.427474
7,20.358648,20.087496,-11.474121,-12.390013
8,-49.958988,-50.566116,-0.835833,-0.404144


## Cluster Tagging

Finally we can investigate if a cluster contributed to the main or alternative S1 or S2. This is done by the `ClusterTagging` plugin. It will provide the following columns:
- `in_main_s1` - Boolean if the cluster contributed to the main S1
- `in_main_s2` - Boolean if the cluster contributed to the main S2
- `in_alt_s1` - Boolean if the cluster contributed to an alternative S1
- `in_alt_s2` - Boolean if the cluster contributed to an alternative S2

In [23]:
st.make(run_number, "tagged_clusters")

We can load it together with e.g. microphysics_summary:

In [24]:
ms_with_tagged_clusters = st.get_df(run_number, ["microphysics_summary", "tagged_clusters"])

Loading plugins: |          | 0.00 % [00:00<?]

Lets take a look at the clusters that contributed to the main S2 of the second event: 

In [25]:
ms_with_tagged_clusters.query("evtid == 1 & in_main_s2 == True").head(10)

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,...,xe_density,vol_id,create_S2,photons,electrons,excitons,in_main_s1,in_main_s2,in_alt_s1,in_alt_s2
60,24,2827386343,2827386343,-12.797632,17.414379,-36.664551,119.665840,8,0,0,...,2.862,1,True,6734,2189,1324,True,True,False,False
61,24,2827386343,2827386343,-12.796987,17.403460,-36.681953,70.190887,8,0,0,...,2.862,1,True,4360,904,833,True,True,False,False
62,24,2827386343,2827386343,-12.808017,17.383905,-36.684723,64.250038,8,0,0,...,2.862,1,True,3942,792,711,True,True,False,False
63,24,2827386343,2827386343,-12.822882,17.374733,-36.682594,75.890457,8,0,0,...,2.862,1,True,4646,969,891,True,True,False,False
64,24,2827386343,2827386343,-12.833847,17.377735,-36.667599,60.865395,8,0,0,...,2.862,1,True,3903,655,696,True,True,False,False
65,24,2827386343,2827386343,-12.837183,17.390974,-36.659126,59.640846,8,0,0,...,2.862,1,True,3768,628,665,True,True,False,False
66,27,2827386343,2827386343,-36.483269,-21.714609,-9.259213,42.037697,8,0,0,...,2.862,1,True,2372,590,482,True,True,False,False
67,27,2827386343,2827386343,-36.388309,-21.533461,-9.131524,183.596298,12,127,127,...,2.862,1,True,10444,3265,2083,True,True,False,False
68,27,2827386343,2827386343,-36.287651,-21.658583,-9.301582,21.843777,8,0,0,...,2.862,1,True,1253,358,235,True,True,False,False
69,27,2827386343,2827386343,-36.293476,-21.664663,-9.303707,32.598518,8,0,0,...,2.862,1,True,1911,492,368,True,True,False,False
